In [1]:
import datetime  
# import sunrise
import calendar
import pandas as pd
import numpy as np

In [3]:
df_crosswalk = pd.read_csv("crosswalk_stata_version.tab",sep='\t')

In [5]:
df_crosswalk = df_crosswalk.dropna(subset=['LONG','LAT'])

In [6]:
list_lat = list(df_crosswalk['LAT'])
list_long = list(df_crosswalk['LONG'])
list_long = [-x if x > 0 else x for x in list_long]

In [2]:
"""Calculate the sunrise, sunset and noon time for a given coordinate.
Based on the code at: https://michelanders.blogspot.com/2010/12/calulating-sunrise-and-sunset-in-python.html
"""
from math import cos, sin, acos, asin, tan
from math import degrees as deg, radians as rad
from datetime import datetime, time, timezone, timedelta


class Sun:
    """
    Calculate sunrise and sunset based on equations from NOAA
    http://www.srrb.noaa.gov/highlights/sunrise/calcdetails.html
    """

    def __init__(self, tz, lat, long):  # default Amsterdam
        self.lat = lat
        self.long = long
        self.when = tz
        self.__preptime(self.when)
        self.__calc()

    def sunrise(self):
        """
        return the time of sunrise as a datetime.time object
        when is a datetime.datetime object. If none is given
        a local time zone is assumed (including daylight saving
        if present)
        """
        return Sun.__timefromdecimalday(self.sunrise_t)

    def sunset(self):
        return Sun.__timefromdecimalday(self.sunset_t)

    def solarnoon(self):
        return Sun.__timefromdecimalday(self.solarnoon_t)

    @staticmethod
    def __timefromdecimalday(day):
        """
        returns a datetime.time object.
        day is a decimal day between 0.0 and 1.0, e.g. noon = 0.5
        """
        hours = 24.0 * day 
        h = int(hours)
        minutes = (hours - h) * 60
        m = int(minutes)
        seconds = (minutes - m) * 60
        s = int(seconds)
        h = h-8
        if h < 0:
            h = 0
            day_no = round(h/24)
            day_no = -1 * day_no
        if 0 <= h < 24:
            day_no = 0
        else:
            h = h - 24
            day_no = round(h/24)
        return (time(hour=h, minute=m, second=s), day_no)

    def __preptime(self, when):
        """
        Extract information in a suitable format from when,
        a datetime.datetime object.
        """
        # datetime days are numbered in the Gregorian calendar
        # while the calculations from NOAA are distibuted as
        # OpenOffice spreadsheets with days numbered from
        # 1/1/1900. The difference are those numbers taken for
        # 18/12/2010
        self.day = when.toordinal() - (734124 - 40529)
        t = when.time()
        self.time = (t.hour + t.minute / 60.0 + t.second / 3600.0) / 24.0

        self.timezone = 0
        offset = when.utcoffset()
        if offset is not None:
            self.timezone = offset.seconds / 3600.0

    def __calc(self):
        """
        Perform the actual calculations for sunrise, sunset and
        a number of related quantities.
        The results are stored in the instance variables
        sunrise_t, sunset_t and solarnoon_t
        """
        timezone = self.timezone  # in hours, east is positive
        longitude = self.long     # in decimal degrees, east is positive
        latitude = self.lat      # in decimal degrees, north is positive

        time = self.time  # percentage past midnight, i.e. noon  is 0.5
        day = self.day     # daynumber 1=1/1/1900

        Jday = day + 2415018.5 + time - timezone / 24  # Julian day
        Jcent = (Jday - 2451545) / 36525    # Julian century

        Manom = 357.52911 + Jcent * (35999.05029 - 0.0001537 * Jcent)
        Mlong = 280.46646 + Jcent * (36000.76983 + Jcent * 0.0003032) % 360
        Eccent = 0.016708634 - Jcent * (0.000042037 + 0.0001537 * Jcent)
        Mobliq = 23 + (26 + ((21.448 - Jcent * (46.815 + Jcent * \
                       (0.00059 - Jcent * 0.001813)))) / 60) / 60
        obliq = Mobliq + 0.00256 * cos(rad(125.04 - 1934.136 * Jcent))
        vary = tan(rad(obliq / 2)) * tan(rad(obliq / 2))
        Seqcent = sin(rad(Manom)) * (1.914602 - Jcent * (0.004817 + 0.000014 * Jcent)) + \
            sin(rad(2 * Manom)) * (0.019993 - 0.000101 * Jcent) + sin(rad(3 * Manom)) * 0.000289
        Struelong = Mlong + Seqcent
        Sapplong = Struelong - 0.00569 - 0.00478 * \
            sin(rad(125.04 - 1934.136 * Jcent))
        declination = deg(asin(sin(rad(obliq)) * sin(rad(Sapplong))))

        eqtime = 4 * deg(vary * sin(2 * rad(Mlong)) - 2 * Eccent * sin(rad(Manom)) + 4 * Eccent * vary * sin(rad(Manom))
                         * cos(2 * rad(Mlong)) - 0.5 * vary * vary * sin(4 * rad(Mlong)) - 1.25 * Eccent * Eccent * sin(2 * rad(Manom)))
        hourangle_pre = (cos(rad(90.833)) /
                             (cos(rad(latitude)) *
                              cos(rad(declination))) -
                             tan(rad(latitude)) *
                             tan(rad(declination)))
        if -1 <= hourangle_pre <= 1:
            hourangle = deg(acos(hourangle_pre))
        else:
            hourangle = 0
#         elif hourangle_pre > 1:
#             hourangle = deg(acos(1))
#         elif hourangle_pre < -1:
#             hourangle = deg(acos(-1))

        self.solarnoon_t = (720 - 4 * longitude - eqtime + timezone * 60) / 1440
        if hourangle == 0:
            self.sunrise_t = 0
            self.sunset_t = 0
        else:
            self.sunrise_t = self.solarnoon_t - hourangle * 4 / 1440
            self.sunset_t = self.solarnoon_t + hourangle * 4 / 1440

In [7]:
if __name__ == "__main__":
    list_rise_set = []
    for n in range(len(list_long)):
        n = n+1
        lat = list_lat[n]
        long = list_long[n]
        for i in range(2005, 2017): # year
            for j in range(1,13): # month
                number_of_days = calendar.monthrange(i,j)[1] +1
                for k in range(1, number_of_days):
                    mytz = datetime(i, j, k, 1, 22, 51, 694536, tzinfo=timezone(timedelta(seconds=7200)))
                    s = Sun(mytz, lat, long)
                    srise = s.sunrise()
                    sset = s.sunset()
                    list_rise_set.append([lat, long, k, j, i, srise[0], srise[1], sset[0], sset[1]])

IndexError: list index out of range

In [8]:
df_rise_set = pd.DataFrame(list_rise_set, columns=['latitude', 'longitude','day', 'month', 'year', 'sunrise',
                                                   'sunrise_day', 'sunset', 'sunset_day'])
df_rise_set.to_csv("sunrise_data.csv")
df_rise_set

,latitude,longitude,day,month,year,sunrise,sunrise_day,sunset,sunset_day
0,32.459133,-86.4513,1,1,2005,06:47:14,0,16:50:15,0
1,32.459133,-86.4513,2,1,2005,06:47:28,0,16:50:57,0
2,32.459133,-86.4513,3,1,2005,06:47:41,0,16:51:41,0
3,32.459133,-86.4513,4,1,2005,06:47:51,0,16:52:26,0
4,32.459133,-86.4513,5,1,2005,06:48:00,0,16:53:12,0
5,32.459133,-86.4513,6,1,2005,06:48:08,0,16:53:59,0
6,32.459133,-86.4513,7,1,2005,06:48:13,0,16:54:46,0
7,32.459133,-86.4513,8,1,2005,06:48:17,0,16:55:35,0
8,32.459133,-86.4513,9,1,2005,06:48:19,0,16:56:24,0
9,32.459133,-86.4513,10,1,2005,06:48:19,0,16:57:14,0


In [1]:
import pandas as pd
df_rise_set = pd.read_csv("sunrise_data.csv")

In [2]:
df_rise_set

,Unnamed: 0,latitude,longitude,day,month,year,sunrise,sunrise_day,sunset,sunset_day
0,0,32.459133,-86.4513,1,1,2005,06:47:14,0,16:50:15,0
1,1,32.459133,-86.4513,2,1,2005,06:47:28,0,16:50:57,0
2,2,32.459133,-86.4513,3,1,2005,06:47:41,0,16:51:41,0
3,3,32.459133,-86.4513,4,1,2005,06:47:51,0,16:52:26,0
4,4,32.459133,-86.4513,5,1,2005,06:48:00,0,16:53:12,0
5,5,32.459133,-86.4513,6,1,2005,06:48:08,0,16:53:59,0
6,6,32.459133,-86.4513,7,1,2005,06:48:13,0,16:54:46,0
7,7,32.459133,-86.4513,8,1,2005,06:48:17,0,16:55:35,0
8,8,32.459133,-86.4513,9,1,2005,06:48:19,0,16:56:24,0
9,9,32.459133,-86.4513,10,1,2005,06:48:19,0,16:57:14,0


In [6]:
df_2016 = df_dropped[df_dropped['year']==2016]

In [8]:
df_2016.to_csv("latlong_sunrise_2016.csv")

In [3]:
df_dropped = df_rise_set.drop(columns=['Unnamed: 0','sunrise_day','sunset_day'])

In [40]:
df_dropped = df_dropped[df_dropped['year'] != 2005]

In [41]:
df_dropped = df_dropped[df_dropped['year'] != 2006]

In [43]:
df_dropped.to_csv("latlong_sunrise.csv")

In [45]:
df_2007 = df_dropped[df_dropped['year']==2007]

In [46]:
df_2007.to_csv("latlong_sunrise_2007.csv")

In [48]:
df_2008 = df_dropped[df_dropped['year']==2008]

In [49]:
df_2008.to_csv("latlong_sunrise_2008.csv")

In [50]:
df_2009 = df_dropped[df_dropped['year']==2009]

In [51]:
df_2009.to_csv("latlong_sunrise_2009.csv")

In [52]:
df_2010 = df_dropped[df_dropped['year']==2010]

In [53]:
df_2010.to_csv("latlong_sunrise_2010.csv")

In [54]:
df_2011 = df_dropped[df_dropped['year']==2011]

In [55]:
df_2011.to_csv("latlong_sunrise_2011.csv")

In [56]:
df_2012 = df_dropped[df_dropped['year']==2012]

In [57]:
df_2012.to_csv("latlong_sunrise_2012.csv")

In [58]:
df_2013 = df_dropped[df_dropped['year']==2013]

In [59]:
df_2013.to_csv("latlong_sunrise_2013.csv")

In [60]:
df_2014 = df_dropped[df_dropped['year']==2014]

In [61]:
df_2014.to_csv("latlong_sunrise_2014.csv")

In [62]:
df_2015 = df_dropped[df_dropped['year']==2015]

In [63]:
df_2015.to_csv("latlong_sunrise_2015.csv")

In [45]:
df_2007 = df_dropped[df_dropped['year']==2016]

In [46]:
df_2007.to_csv("latlong_sunrise_2016.csv")

In [9]:
df_2005 = df_dropped[df_dropped['year']==2005]

In [10]:
df_2005.to_csv("latlong_sunrise_2005.csv")

In [11]:
df_2006 = df_dropped[df_dropped['year']==2006]

In [12]:
df_2006.to_csv("latlong_sunrise_2006.csv")